$$
    \newcommand{\prover}{\mathcal{P}}
    \newcommand{\verifier}{\mathcal{V}}
    \newcommand{\Mat}[1]{\mathbf{{#1}}}
$$

# Spartan: Efficient and general-purpose zkSNARKs without trusted setup

Authors: Srinath Setty <br/>
ePrint: [2019/550](https://eprint.iacr.org/2019/550.pdf)

## Introduction

Spartan is a transparent [SNARK](https://en.wikipedia.org/wiki/Non-interactive_zero-knowledge_proof) for R1CS [constraint satisfaction problem](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem). R1CS is a popular NP-Complete problem that enjoys extensive tooling support from the compiler community (or so I've been told).

An R1CS instance consists of
1. Three $n \times m$ _sparse_ matrices $A, B, C \in \mathbb{F}^{n\times m}$ that encode the circuit/constraints representing the CSP, and 
2. An arbitrary $m$ dimensional vector $\vec{w} \in \mathbb{F}^m$.

The vector $\vec{w}$ is _considered a witness_ to the NP statement if it satisfies the following relation:

$$
    (A\cdot \vec{w}) {\color{red} \circ }(B\cdot \vec{w}) = C\cdot \vec{w}
$$

where ${\color{red} \circ }$ is the Hadamard (element wise) product. (R1CS captures arithmetic circuits with fan-in $2$ multiplication gates, and arbitrary fan-in addition gates.)

Without loss of generality, we assume $m = n$ in the rest of the document and treat $A,B,C \in \mathbb{F}^{m\times m}$, and $w \in \mathbb{F}^m$ (if $m\neq n$, then $A, B, C,$ and $w$ can be padded appropriately to get the desired result). Furthermore, a matrix $M \in \mathbb{F}^{m\times m}$ is defined to be sparse if the number of non-zero entries, call it $\ell$, is sublinear in $m^2$(for example, say $\ell \in \Theta(m)$).

Spartan -- as described here -- is the Polynomial Interactive Oracle Proof (PIOP) that can be converted into a SNARK using any suitable polynomial commitment scheme (PCS). 

The remarkable features of Spartan PIOP are:
* the prover's time is _strictly linear_ in $\ell$, and 
* the verifier time is polylogrithmic in $\ell$, i.e., $O(\log(\ell)^c)$ for some fixed constant $c$. 

Note that in a typical use case (e.g. google's
[zk-longfellow](https://github.com/google/longfellow-zk.git) for legacy identity
verification), the value of $\ell$ is of the order of $2^{25}$. Unless the
prover time is linear (or at worst quasi-linear) in $\ell$, it won't be
possible to build a useful system.

### Sumcheck: A protocol for $\forall \to \exists$ quantifier reduction

The goal of _any_ R1CS prover is to prove that _for all_ rows: $(A\cdot w)\circ(B\cdot w) - (C\cdot w) = 0$ holds. However, since the verifier's time is required to be poly-logarithmic in the witness length $m := |w|$, it cannot check the validity of each row individually! Therefore, the fist design choice in building any R1CS SNARK is to decide how to transform a _logical claim_ about "**for all** rows" to a claim about "**for only** a polylog number rows," without sacrificing too much in soundness.

The simplest strategy one could think of is that the verifier _randomly_ checks a polylog number of rows, say $\log(m)^{\kappa}$ number of rows for some integer $\kappa$. This strategy, however, allows a malicious prover to cheat with probability $1 - \frac{\log(m)^\kappa}{m} \approx 1 - o(\frac{1}{m})$ (for large $m$). This is unsatisfactory because for large values of $m$ (say $m \approx 2^{20}$), this strategy allows the prover to cheat with almost certainty! Ideally, one would like the cheating probability to  grow -- at best -- poly-logarithmically in the size of the problem instance (ideally size of witness), and be adjustable according to some security parameters $\lambda$.

Sumcheck [[LFKN'92](https://lance.fortnow.com/papers/files/ip.pdf)] is an elegant protocol for _reducing_ a claim about the _sum_ of **all** evaluations of a multivariate polynomial over a hypercube, to a claim about **one** random evaluation by the verifier. In particular if $f(\vec{x}) \in \mathbb{F}^{(\preceq d)}[x_1,\cdots, x_\mu]$ is a $\mu$-variate polynomial with maximal individual degree $d$, then Sumcheck is a reduction from a claim of the form 

$$
F \stackrel{?}{=} \sum_{\vec{b} \in \{0,1\}^\mu} f(\vec{b})
$$

to a claim of the form 

$$
G \stackrel{?}{=} f(r_1,\cdots, r_\mu)
$$

The soundness loss (i.e., the probability for prover to cheat) in the above reduction is $1 - (1 - \frac{d}{|\mathbb{F}|})^\mu \approx \mu \cdot \frac{d}{|\mathbb{F}|}$.

For an R1CS instance of size $m$, Spartan tries to reduce the R1CS claim to a Sumcheck instance claim, where $\mu = O(\log |m|)$ and the individual degree of polynomials in the Sumcheck instance is $d \leq 3$. 

### Spartan Attempt-0


![Spartan Attempt 0](./spartan/Spartan-0.drawio.svg)

### Spartan Full
![Spartan Sumcheck-1](./spartan/Spartan-Full-SCHK-1.drawio.svg)

![Spartan Sumcheck-2](./spartan/Spartan-Full-SCHK-2.drawio.svg)

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('./sage-snark'))
sys.path.insert(0, module_path)

In [2]:
from sage.matrix.all import Matrix
from sage.rings.integer import Integer
from sage.rings.polynomial.all import PolynomialRing, Polynomial
from sage.modules.all import vector

from utils.multivariates import hadamard_product, hypercube_sum, bit_decomp_dict, matrix_multilinearize, vec_multilinearize,
from utils.test_utils import R1CS

SyntaxError: trailing comma not allowed without surrounding parentheses (3420249138.py, line 6)

In [ ]:
# Sample R1CS Instance. Taken from: https://emirsoyturk.medium.com/hello-arithmetization-55e57c8e5471

AL = [
     [0, 1, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0],
     [0, 1, 0, 0, 1, 0],
     [5, 0, 0, 0, 0, 1]
    ]

BL = [
     [0, 1, 0, 0, 0, 0],
     [0, 1, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0]
    ]

CL = [
     [0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 1],
     [0, 0, 1, 0, 0, 0]
    ]

WL = [1, 3, 35, 9, 27, 30]

In [ ]:
def compute_y_sum(poly, y_dim):
    gens = poly.parent().gens()
    skip_len = len(gens) - Integer(y_dim).bit_length()
    assert skip_len >= 0

    skip_vars = gens[:skip_len]
    return hypercube_sum(poly, skip_vars)

In [ ]:
P = 15*(2**27) + 1;
assert is_prime(P)
Fp = GF(P)

A = Matrix(Fp, AL)
B = Matrix(Fp, BL)
C = Matrix(Fp, CL)
w = vector(Fp, WL);
aw = A*w;
bw = B*w;
cw = C*w;

# print(f"W = {w}")
# print(f"A*W = {aw}")
# print(f"B*W = {bw}")
# print(f"C*W = {cw}")

assert list(cw) == hadamard_product(aw, bw)

In [ ]:
Axy = matrix_multilinearize(A);
Bxy = matrix_multilinearize(B, Axy.parent())
Cxy = matrix_multilinearize(C, Axy.parent())

xgens_count = Integer(A.nrows()).bit_length()
xgens = Axy.parent().gens()[:xgens_count];
ygens = Axy.parent().gens()[xgens_count:];

Wy = vec_multilinearize(w, ygens)

Ax = compute_y_sum(Axy*Wy, 6)
Bx = compute_y_sum(Bxy*Wy, 6)
Cx = compute_y_sum(Cxy*Wy, 6)

expected = Ax*Bx - Cx

for i in range(A.nrows()):
    d = bit_decomp_dict(i, xgens)
    e = expected.subs(d)
    assert e == 0
